In [2]:
#cifar-10-test
import numpy as np
import pandas as pd
import scipy
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn import metrics
from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials,rand
from hyperopt.mongoexp import MongoTrials
import h5py
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import sobol
import datetime
import os
import math
from __future__ import absolute_import,division,print_function
import argparse
import sys
import time
from pyDOE import *
from sklearn.decomposition import PCA
import cv2

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:

def readData(pairpathlabel):
    '''read image to list'''
    imgs = []
    labels = []
    for filepath, label in pairpathlabel:
        for (path,dirnames,filenames) in os.walk(filepath):
            print(path)
            for filename in filenames:
                print(filename)
                if filename.endswith('.jpg') or filename.endswith('.PNG'):
                    img_path=path+'/'+filename
                    img=cv2.imread(img_path,0)
                    #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    imgs.append(img)
                    labels.append(label)
    return np.array(imgs), np.array(labels)
def onehot(numlist):
    ''' get one hot return host matrix is len * max+1 demensions'''
    b = np.zeros([len(numlist), max(numlist)+1])
    b[np.arange(len(numlist)), numlist] = 1
    return b.tolist()

def getfileandlabel(filedir):
    ''' get path and host paire and class index to name'''
    dictdir = dict([[name, os.path.join(filedir, name)] \
                    for name in os.listdir(filedir) 
                    if os.path.isdir(os.path.join(filedir, name)) and not name.startswith('.')])
                    #for (path, dirnames, _) in os.walk(filedir) for dirname in dirnames])
    dirnamelist, dirpathlist = dictdir.keys(), dictdir.values()
    indexlist = list(range(len(dirnamelist)))
    return list(zip(dirpathlist, (indexlist))), dict(zip(indexlist, dirnamelist))
    
pathlabelpair, indextoname = getfileandlabel('images')
x, y = readData(pathlabelpair)
print(np.array(x).shape,y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.5,stratify =y, random_state=0)

imgs=[]
labels=[]
size=90
for (img,label) in zip(X_train,y_train):
    rows,cols = img.shape[:2]
    for i in range(rows-size):
        for j in range(cols-size):
            imgs.append(img[i:i+size,j:j+size])
            labels.append(label)
            imgs.append(cv2.flip(img[i:i+size,j:j+size],1))
            labels.append(label)
            imgs.append(cv2.flip(img[i:i+size,j:j+size],0))
            labels.append(label)
            imgs.append(cv2.flip(img[i:i+size,j:j+size],-1))  
            labels.append(label)
            clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
            imgs.append(clahe.apply(img[i:i+size,j:j+size]))
            labels.append(label)
            imgs.append(clahe.apply(cv2.flip(img[i:i+size,j:j+size],1)))
            labels.append(label)
            imgs.append(clahe.apply(cv2.flip(img[i:i+size,j:j+size],0)))
            labels.append(label)
            imgs.append(clahe.apply(cv2.flip(img[i:i+size,j:j+size],-1)))
            labels.append(label)
            

    print(np.array(imgs).shape,"+++",np.array(labels).shape)
imgs=np.array(imgs)
labels=np.array(labels)
X_train, X_val, y_train, y_val = train_test_split(imgs,labels, test_size=.3,stratify =labels, random_state=0)
         
te_imgs_1=[]
te_labels_1=[]
te_imgs_h1=[]
te_labels_h1=[]
te_imgs_v1=[]
te_labels_v1=[]
te_imgs_hv1=[]
te_labels_hv1=[]
te_imgs_2=[]
te_labels_2=[]
te_imgs_h2=[]
te_labels_h2=[]
te_imgs_v2=[]
te_labels_v2=[]
te_imgs_hv2=[]
te_labels_hv2=[]
te_imgs_3=[]
te_labels_3=[]
te_imgs_h3=[]
te_labels_h3=[]
te_imgs_v3=[]
te_labels_v3=[]
te_imgs_hv3=[]
te_labels_hv3=[]
te_imgs_4=[]
te_labels_4=[]
te_imgs_h4=[]
te_labels_h4=[]
te_imgs_v4=[]
te_labels_v4=[]
te_imgs_hv4=[]
te_labels_hv4=[]
te_imgs_5=[]
te_labels_5=[]
te_imgs_h5=[]
te_labels_h5=[]
te_imgs_v5=[]
te_labels_v5=[]
te_imgs_hv5=[]
te_labels_hv5=[]

size=90
for (img,label) in zip(X_test,y_test): 
    h,w = img.shape[:2]
    label=label-1
    for i in range(1,6):                            
        if i==1:
            te_imgs_1.append(img[0:size,0:size])
            te_labels_1.append(label)
            te_imgs_h1.append(cv2.flip(img[0:size,0:size],1))
            te_labels_h1.append(label)
            te_imgs_v1.append(cv2.flip(img[0:size,0:size],0))
            te_labels_v1.append(label)
            te_imgs_hv1.append(cv2.flip(img[0:size,0:size],-1)) 
            te_labels_hv1.append(label)
        elif i==2:
            te_imgs_2.append(img[h-size:h,0:size])
            te_labels_2.append(label)
            te_imgs_h2.append(cv2.flip(img[h-size:h,0:size],1))
            te_labels_h2.append(label)
            te_imgs_v2.append(cv2.flip(img[h-size:h,0:size],0))
            te_labels_v2.append(label)
            te_imgs_hv2.append(cv2.flip(img[h-size:h,0:size],-1))  
            te_labels_hv2.append(label)
        elif i==3:
            te_imgs_3.append(img[h-size:h,w-size:w])
            te_labels_3.append(label)
            te_imgs_h3.append(cv2.flip(img[h-size:h,w-size:w],1))
            te_labels_h3.append(label)
            te_imgs_v3.append(cv2.flip(img[h-size:h,w-size:w],0))
            te_labels_v3.append(label)
            te_imgs_hv3.append(cv2.flip(img[h-size:h,w-size:w],-1)) 
            te_labels_hv3.append(label)
        elif i==4:
            te_imgs_4.append(img[0:size,w-size:w])
            te_labels_4.append(label)
            te_imgs_h4.append(cv2.flip(img[0:size,w-size:w],1))
            te_labels_h4.append(label)
            te_imgs_v4.append(cv2.flip(img[0:size,w-size:w],0))
            te_labels_v4.append(label)
            te_imgs_hv4.append(cv2.flip(img[0:size,w-size:w],-1)) 
            te_labels_hv4.append(label)
        else :
            hs=(h-size)//2
            ws=(w-size)//2
            te_imgs_5.append(img[hs:hs+size,ws:ws+size])
            te_labels_5.append(label)
            te_imgs_h5.append(cv2.flip(img[hs:hs+size,ws:ws+size],1))
            te_labels_h5.append(label)
            te_imgs_v5.append(cv2.flip(img[hs:hs+size,ws:ws+size],0))
            te_labels_v5.append(label)
            te_imgs_hv5.append(cv2.flip(img[hs:hs+size,ws:ws+size],-1)) 
            te_labels_hv5.append(label)


images/cluster
defect_test1.PNG
defect_test.PNG
images/ball
defect_test21.PNG
defect_test2.PNG
(4,) (4,)
(192, 90, 90) +++ (192,)
(1512, 90, 90) +++ (1512,)


In [18]:
np.array(X_test).shape

(2,)

In [21]:
space = {
    'momentum': hp.uniform('momentum', 0.1,0.95),   
    'learning_rate': hp.uniform('learning_rate', 0.0001,0.1),
    'weight_decay': hp.uniform('weight_decay', 0.00001,0.01),
    'batch_size' : hp.choice('batch_size', range(28,256))
       }
def SE_Inception_resnet_v2_1(params):
    import tensorflow as tf
    #from tflearn.layers.conv import global_avg_pool
    from tensorflow.contrib.layers import batch_norm, flatten
    from tensorflow.contrib.framework import arg_scope
    #from cifar10 import *
    import numpy as np
    import os
    import sys
    import time
    import pickle
    import random

    class_num = 2
    image_size = 90
    img_channels = 1
    weight_decay = params['weight_decay']
    momentum =params['momentum']
    init_learning_rate = params['learning_rate']
    reduction_ratio = 16
    batch_size = params['batch_size']
    
    def conv_layer(input, filter, kernel, stride=1, padding='SAME', layer_name="conv", activation=True):
        with tf.name_scope(layer_name):
            network = tf.layers.conv2d(inputs=input, use_bias=True, filters=filter, kernel_size=kernel, strides=stride, padding=padding)
            if activation :
                network = Relu(network)
            return network

    def Fully_connected(x, units=class_num, layer_name='fully_connected') :
        with tf.name_scope(layer_name) :
            return tf.layers.dense(inputs=x, use_bias=True, units=units)

    def Relu(x):
        return tf.nn.relu(x)

    def Sigmoid(x):
        return tf.nn.sigmoid(x)

    def get_incoming_shape(incoming):
        """ Returns the incoming data shape """
        if isinstance(incoming, tf.Tensor):
            return incoming.get_shape().as_list()
        elif type(incoming) in [np.array, np.ndarray, list, tuple]:
            return np.shape(incoming)
        else:
            raise Exception("Invalid incoming layer.")

    def global_avg_pool(incoming, name="GlobalAvgPool"):
        input_shape = get_incoming_shape(incoming)
        assert len(input_shape) == 4, "Incoming Tensor shape must be 4-D"
        with tf.name_scope(name):
            inference = tf.reduce_mean(incoming, [1, 2])
        return inference

    def Global_Average_Pooling(x):
        return global_avg_pool(x, name='Global_avg_pooling')

    def Max_pooling(x, pool_size=[3,3], stride=2, padding='VALID') :
        return tf.layers.max_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)

    def Batch_Normalization(x, training, scope):
        with arg_scope([batch_norm],
                       scope=scope,
                       updates_collections=None,
                       decay=0.9,
                       center=True,
                       scale=True,
                       zero_debias_moving_mean=True) :
            return tf.cond(training,
                           lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                           lambda : batch_norm(inputs=x, is_training=training, reuse=True))

    def Concatenation(layers) :
        return tf.concat(layers, axis=3)

    def Dropout(x, rate, training) :
        return tf.layers.dropout(inputs=x, rate=rate, training=training)

    def Evaluate(sess):
        val_acc = 0.0
        val_loss = 0.0

        test_feed_dict = {
            x: X_val,
            label: y_val,
            learning_rate: epoch_learning_rate,
            training_flag: False
        }

        loss_, acc_ = sess.run([cost, accuracy], feed_dict=test_feed_dict)

        val_loss += loss_
        val_acc += acc_
        return val_acc, val_loss

    class SE_Inception_resnet_v2():
        def __init__(self, x, training):
            self.training = training
            self.model = self.Build_SEnet(x)

        def Stem(self, x, scope):
            with tf.name_scope(scope) :
                x = conv_layer(x, filter=32, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_conv1')
                x = conv_layer(x, filter=32, kernel=[3,3], padding='VALID', layer_name=scope+'_conv2')
                block_1 = conv_layer(x, filter=64, kernel=[3,3], layer_name=scope+'_conv3')

                split_max_x = Max_pooling(block_1)
                split_conv_x = conv_layer(block_1, filter=96, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv1')
                x = Concatenation([split_max_x,split_conv_x])

                split_conv_x1 = conv_layer(x, filter=64, kernel=[1,1], layer_name=scope+'_split_conv2')
                split_conv_x1 = conv_layer(split_conv_x1, filter=96, kernel=[3,3], padding='VALID', layer_name=scope+'_split_conv3')

                split_conv_x2 = conv_layer(x, filter=64, kernel=[1,1], layer_name=scope+'_split_conv4')
                split_conv_x2 = conv_layer(split_conv_x2, filter=64, kernel=[7,1], layer_name=scope+'_split_conv5')
                split_conv_x2 = conv_layer(split_conv_x2, filter=64, kernel=[1,7], layer_name=scope+'_split_conv6')
                split_conv_x2 = conv_layer(split_conv_x2, filter=96, kernel=[3,3], padding='VALID', layer_name=scope+'_split_conv7')

                x = Concatenation([split_conv_x1,split_conv_x2])

                split_conv_x = conv_layer(x, filter=192, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv8')
                split_max_x = Max_pooling(x)

                x = Concatenation([split_conv_x, split_max_x])

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Inception_resnet_A(self, x, scope):
            with tf.name_scope(scope) :
                init = x

                split_conv_x1 = conv_layer(x, filter=32, kernel=[1,1], layer_name=scope+'_split_conv1')

                split_conv_x2 = conv_layer(x, filter=32, kernel=[1,1], layer_name=scope+'_split_conv2')
                split_conv_x2 = conv_layer(split_conv_x2, filter=32, kernel=[3,3], layer_name=scope+'_split_conv3')

                split_conv_x3 = conv_layer(x, filter=32, kernel=[1,1], layer_name=scope+'_split_conv4')
                split_conv_x3 = conv_layer(split_conv_x3, filter=48, kernel=[3,3], layer_name=scope+'_split_conv5')
                split_conv_x3 = conv_layer(split_conv_x3, filter=64, kernel=[3,3], layer_name=scope+'_split_conv6')

                x = Concatenation([split_conv_x1,split_conv_x2,split_conv_x3])
                x = conv_layer(x, filter=384, kernel=[1,1], layer_name=scope+'_final_conv1', activation=False)

                x = x*0.1
                x = init + x

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Inception_resnet_B(self, x, scope):
            with tf.name_scope(scope) :
                init = x

                split_conv_x1 = conv_layer(x, filter=192, kernel=[1,1], layer_name=scope+'_split_conv1')

                split_conv_x2 = conv_layer(x, filter=128, kernel=[1,1], layer_name=scope+'_split_conv2')
                split_conv_x2 = conv_layer(split_conv_x2, filter=160, kernel=[1,7], layer_name=scope+'_split_conv3')
                split_conv_x2 = conv_layer(split_conv_x2, filter=192, kernel=[7,1], layer_name=scope+'_split_conv4')

                x = Concatenation([split_conv_x1, split_conv_x2])
                x = conv_layer(x, filter=1152, kernel=[1,1], layer_name=scope+'_final_conv1', activation=False)
                # 1154
                x = x * 0.1
                x = init + x

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Inception_resnet_C(self, x, scope):
            with tf.name_scope(scope) :
                init = x

                split_conv_x1 = conv_layer(x, filter=192, kernel=[1,1], layer_name=scope+'_split_conv1')

                split_conv_x2 = conv_layer(x, filter=192, kernel=[1, 1], layer_name=scope + '_split_conv2')
                split_conv_x2 = conv_layer(split_conv_x2, filter=224, kernel=[1, 3], layer_name=scope + '_split_conv3')
                split_conv_x2 = conv_layer(split_conv_x2, filter=256, kernel=[3, 1], layer_name=scope + '_split_conv4')

                x = Concatenation([split_conv_x1,split_conv_x2])
                x = conv_layer(x, filter=2144, kernel=[1,1], layer_name=scope+'_final_conv2', activation=False)
                # 2048
                x = x * 0.1
                x = init + x

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Reduction_A(self, x, scope):
            with tf.name_scope(scope) :
                k = 256
                l = 256
                m = 384
                n = 384

                split_max_x = Max_pooling(x)

                split_conv_x1 = conv_layer(x, filter=n, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv1')

                split_conv_x2 = conv_layer(x, filter=k, kernel=[1,1], layer_name=scope+'_split_conv2')
                split_conv_x2 = conv_layer(split_conv_x2, filter=l, kernel=[3,3], layer_name=scope+'_split_conv3')
                split_conv_x2 = conv_layer(split_conv_x2, filter=m, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv4')

                x = Concatenation([split_max_x, split_conv_x1, split_conv_x2])

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Reduction_B(self, x, scope):
            with tf.name_scope(scope) :
                split_max_x = Max_pooling(x)

                split_conv_x1 = conv_layer(x, filter=256, kernel=[1,1], layer_name=scope+'_split_conv1')
                split_conv_x1 = conv_layer(split_conv_x1, filter=384, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv2')

                split_conv_x2 = conv_layer(x, filter=256, kernel=[1,1], layer_name=scope+'_split_conv3')
                split_conv_x2 = conv_layer(split_conv_x2, filter=288, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv4')

                split_conv_x3 = conv_layer(x, filter=256, kernel=[1,1], layer_name=scope+'_split_conv5')
                split_conv_x3 = conv_layer(split_conv_x3, filter=288, kernel=[3,3], layer_name=scope+'_split_conv6')
                split_conv_x3 = conv_layer(split_conv_x3, filter=320, kernel=[3,3], stride=2, padding='VALID', layer_name=scope+'_split_conv7')

                x = Concatenation([split_max_x, split_conv_x1, split_conv_x2, split_conv_x3])

                x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
                x = Relu(x)

                return x

        def Squeeze_excitation_layer(self, input_x, out_dim, ratio, layer_name):
            with tf.name_scope(layer_name) :

                squeeze = Global_Average_Pooling(input_x)

                excitation = Fully_connected(squeeze, units=out_dim / ratio, layer_name=layer_name+'_fully_connected1')
                excitation = Relu(excitation)
                excitation = Fully_connected(excitation, units=out_dim, layer_name=layer_name+'_fully_connected2')
                excitation = Sigmoid(excitation)

                excitation = tf.reshape(excitation, [-1,1,1,out_dim])
                scale = input_x * excitation

                return scale

        def Build_SEnet(self, input_x):
            input_x = tf.pad(input_x, [[0, 0], [32, 32], [32, 32], [0, 0]])
            # size 32 -> 96
            print(np.shape(input_x))
            # only cifar10 architecture

            x = self.Stem(input_x, scope='stem')

            for i in range(5) :
                x = self.Inception_resnet_A(x, scope='Inception_A'+str(i))
                channel = int(np.shape(x)[-1])
                x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_A'+str(i))

            x = self.Reduction_A(x, scope='Reduction_A')

            channel = int(np.shape(x)[-1])
            x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_A')

            for i in range(10)  :
                x = self.Inception_resnet_B(x, scope='Inception_B'+str(i))
                channel = int(np.shape(x)[-1])
                x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_B'+str(i))

            x = self.Reduction_B(x, scope='Reduction_B')

            channel = int(np.shape(x)[-1])
            x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_B')

            for i in range(5) :
                x = self.Inception_resnet_C(x, scope='Inception_C'+str(i))
                channel = int(np.shape(x)[-1])
                x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_C'+str(i))

            # channel = int(np.shape(x)[-1])
            # x = self.Squeeze_excitation_layer(x, out_dim=channel, ratio=reduction_ratio, layer_name='SE_C')

            x = Global_Average_Pooling(x)
            x = Dropout(x, rate=0.2, training=self.training)
            x = flatten(x)

            x = Fully_connected(x, layer_name='final_fully_connected')
            return x

    print(params)
    epochs = 15
    train_num_examples=y_train.shape[0]
    max_steps = int(math.ceil(train_num_examples / batch_size))
    total_sample_train = max_steps * batch_size
    best = 0
    min_val_loss = 1
    wait = 0  #counter for patience
    best_rounds =1
    counter=0
    patience=2

    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, shape=[None, image_size, image_size])
    x1 = tf.reshape(x, [-1,90,90,1])
    label = tf.placeholder(tf.int32, shape=[None])
    training_flag = tf.placeholder(tf.bool)
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')

    logits = SE_Inception_resnet_v2(x1, training=training_flag).model
    cost = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(labels=label, logits=logits))

    l2_loss = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum, use_nesterov=True)
    train = optimizer.minimize(cost + l2_loss * weight_decay)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.cast(label,tf.int64))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    top_k_op = tf.nn.in_top_k(logits, label, 1)

    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(y_train.shape[0] * min_fraction_of_examples_in_queue)
    input_queue=tf.train.slice_input_producer([X_train,y_train],shuffle=False)
    X_batch,y_batch = tf.train.batch(input_queue,batch_size=batch_size,num_threads=100,
                                     capacity=min_queue_examples + 3 *batch_size)  

    print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
    # 定义会话并开始迭代训练
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # 启动图片数据增强的线程队列
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    train_acc_all=[]
    train_loss_all=[]
    val_acc_all=[]
    val_loss_all=[]
    epoch_learning_rate = init_learning_rate
    for epoch in xrange(1,epochs+1):
        if epoch % 30 == 0 :
            epoch_learning_rate = epoch_learning_rate / 10
        true_count_train = 0
        pre_index = 0
        train_acc = 0.0
        train_loss = 0.0
        for step in xrange(1,max_steps+1):
            image_batch, label_batch = sess.run([X_batch,y_batch])
            train_feed_dict = {
                x: image_batch,
                label: label_batch,
                learning_rate: epoch_learning_rate,
                training_flag: True
            }

            _, batch_loss = sess.run([train, cost], feed_dict=train_feed_dict)
            batch_acc = accuracy.eval(feed_dict=train_feed_dict)
            train_loss += batch_loss
            train_acc += batch_acc
            pre_index += batch_size

        train_loss /= max_steps # average loss
        train_acc /= max_steps # average accuracy
        train_acc_all.append(train_acc)
        train_loss_all.append(train_loss)

        val_acc, val_loss = Evaluate(sess)
        val_acc_all.append(val_acc)
        val_loss_all.append(val_loss)

        counter +=1
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            best = val_acc
            best_rounds=counter
            wait = 0
            if not os.path.exists('./model'):
                os.makedirs('./model')
            saver=tf.train.Saver()
            saver.save(sess,save_path='./model/Inception_resnet_v2.ckpt')
        else:
            wait += 1 #incremental the number of times without improvement

        line = "epoch: %d/%d, train_loss: %f, train_acc: %f, val_loss: %f, val_acc: %f \n" % (
            epoch, epochs, train_loss, train_acc, val_loss, val_acc)
        with open('logs.txt', 'a') as f:
            f.write(line)
            
        if wait >= patience : #no more patience, retrieve best model
            break
            
    print('epoch--:%d,best_rounds: %d, min_val_loss: %f, best_val_acc: %f ,best_train_acc: %f' % 
          (epoch,best_rounds,min_val_loss, best,train_acc_all[best_rounds-1]))
    
    try: 
        saver
    except NameError:
        print ('saver_exists=False')
    else:
        saver.restore(sess,save_path='./model/Inception_resnet_v2.ckpt')   
    print('####################################')
    print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
    te_1_logits=logits.eval(feed_dict={x: te_imgs_1,label: te_labels_1,learning_rate: epoch_learning_rate,training_flag: False})
    te_h1_logits=logits.eval(feed_dict={x: te_imgs_h1,label: te_labels_h1,learning_rate: epoch_learning_rate,training_flag: False})
    te_v1_logits=logits.eval(feed_dict={x: te_imgs_v1,label: te_labels_v1,learning_rate: epoch_learning_rate,training_flag: False})
    te_hv1_logits=logits.eval(feed_dict={x: te_imgs_hv1,label: te_labels_hv1,learning_rate: epoch_learning_rate,training_flag: False})
    te_2_logits=logits.eval(feed_dict={x: te_imgs_2,label: te_labels_2,learning_rate: epoch_learning_rate,training_flag: False})
    te_h2_logits=logits.eval(feed_dict={x: te_imgs_h2,label: te_labels_h2,learning_rate: epoch_learning_rate,training_flag: False})
    te_v2_logits=logits.eval(feed_dict={x: te_imgs_v2,label: te_labels_v2,learning_rate: epoch_learning_rate,training_flag: False})
    te_hv2_logits=logits.eval(feed_dict={x: te_imgs_hv2,label: te_labels_hv2,learning_rate: epoch_learning_rate,training_flag: False})
    te_3_logits=logits.eval(feed_dict={x: te_imgs_3,label: te_labels_3,learning_rate: epoch_learning_rate,training_flag: False})
    te_h3_logits=logits.eval(feed_dict={x: te_imgs_h3,label: te_labels_h3,learning_rate: epoch_learning_rate,training_flag: False})
    te_v3_logits=logits.eval(feed_dict={x: te_imgs_v3,label: te_labels_v3,learning_rate: epoch_learning_rate,training_flag: False})
    te_hv3_logits=logits.eval(feed_dict={x: te_imgs_hv3,label: te_labels_hv3,learning_rate: epoch_learning_rate,training_flag: False})
    te_4_logits=logits.eval(feed_dict={x: te_imgs_4,label: te_labels_4,learning_rate: epoch_learning_rate,training_flag: False})
    te_h4_logits=logits.eval(feed_dict={x: te_imgs_h4,label: te_labels_h4,learning_rate: epoch_learning_rate,training_flag: False})
    te_v4_logits=logits.eval(feed_dict={x: te_imgs_v4,label: te_labels_v4,learning_rate: epoch_learning_rate,training_flag: False})
    te_hv4_logits=logits.eval(feed_dict={x: te_imgs_hv4,label: te_labels_hv4,learning_rate: epoch_learning_rate,training_flag: False})
    te_5_logits=logits.eval(feed_dict={x: te_imgs_5,label: te_labels_5,learning_rate: epoch_learning_rate,training_flag: False})
    te_h5_logits=logits.eval(feed_dict={x: te_imgs_h5,label: te_labels_h5,learning_rate: epoch_learning_rate,training_flag: False})
    te_v5_logits=logits.eval(feed_dict={x: te_imgs_v5,label: te_labels_v5,learning_rate: epoch_learning_rate,training_flag: False})
    te_hv5_logits=logits.eval(feed_dict={x: te_imgs_hv5,label: te_labels_hv5,learning_rate: epoch_learning_rate,training_flag: False})

    te_logits =(te_1_logits+te_2_logits+te_3_logits+te_4_logits+te_5_logits+
        te_h1_logits+te_h2_logits+te_h3_logits+te_h4_logits+te_h5_logits+
        te_v1_logits+te_v2_logits+te_v3_logits+te_v4_logits+te_v5_logits+
        te_hv1_logits+te_hv2_logits+te_hv3_logits+te_hv4_logits+te_hv5_logits)/20
    te_ture = sess.run(tf.nn.in_top_k(tf.convert_to_tensor(te_logits), y_test, 1))
    print(te_ture)
    te_true_count = np.sum(te_ture)
    te_accuracy = te_true_count / y_test.shape[0]
    print(te_accuracy)
    coord.request_stop()
    coord.join(threads)
    sess.close()
    
    return {'loss': val_acc*(-1),'test_accuracy':te_accuracy,'val_accuracy':best,'train_accuracy':train_acc_all[best_rounds-1],'best_rounds':best_rounds,'status': STATUS_OK}
    print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
    
print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
max_evals=20
doe_num=10
trials = Trials()
best =  fmin(SE_Inception_resnet_v2_1, space, algo=tpe.suggest, trials=trials, verbose=0, max_evals=max_evals, doe_num=doe_num)
print ("best:",best)
print (trials.best_trial)
print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))

2018-09-28 15:32:39
{'learning_rate': 0.09129130831138302, 'weight_decay': 0.008936897183315345, 'batch_size': 253, 'momentum': 0.2861938245723319}
(?, 154, 154, 1)
2018-09-28 15:33:11
epoch--:3,best_rounds: 1, min_val_loss: 0.653341, best_val_acc: 0.872247 ,best_train_acc: 0.961265
INFO:tensorflow:Restoring parameters from ./model/Inception_resnet_v2.ckpt
####################################
2018-09-28 15:47:09
[ True False]
0.5
{'learning_rate': 0.06709673801330117, 'weight_decay': 0.006727770563230353, 'batch_size': 142, 'momentum': 0.9497824137110675}
(?, 154, 154, 1)
2018-09-28 15:47:51
epoch--:3,best_rounds: 1, min_val_loss: 0.598758, best_val_acc: 0.872247 ,best_train_acc: 0.957746
INFO:tensorflow:Restoring parameters from ./model/Inception_resnet_v2.ckpt
####################################
2018-09-28 16:00:01
[ True False]
0.5
{'learning_rate': 0.08014761554759614, 'weight_decay': 0.0008586781478701359, 'batch_size': 29, 'momentum': 0.6768825652374666}
(?, 154, 154, 1)
2018-09

KeyboardInterrupt: 

In [18]:
print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))
max_evals=20
doe_num=10
trials = Trials()
best =  fmin(SE_Inception_resnet_v2_1, space, algo=tpe.suggest, trials=trials, verbose=0, max_evals=max_evals, doe_num=doe_num)
print ("best:",best)
print (trials.best_trial)
print (time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))

2018-09-12 17:14:10
{'learning_rate': 0.06717472444022282, 'weight_decay': 0.002658981450422523, 'batch_size': 106, 'momentum': 0.5367455912002034}
(?, 154, 154, 3)
2018-09-12 17:14:32
epoch--:1
Best number of rounds: 1,best_train_accuracy: 0.941509, best_val_precision: 0.872247
epoch: 1/15, train_loss: 0.287092, train_acc: 0.941509, val_loss: 0.656459, val_acc: 0.872247 

epoch--:2
Best number of rounds: 1,best_train_accuracy: 0.941509, best_val_precision: 0.872247
epoch: 2/15, train_loss: 0.013249, train_acc: 1.000000, val_loss: 0.652289, val_acc: 0.872247 

epoch--:3
Best number of rounds: 1,best_train_accuracy: 0.941509, best_val_precision: 0.872247
epoch: 3/15, train_loss: 0.005578, train_acc: 1.000000, val_loss: 0.650332, val_acc: 0.872247 

epoch--:4
Best number of rounds: 4,best_train_accuracy: 1.000000, best_val_precision: 1.000000
epoch: 4/15, train_loss: 0.004055, train_acc: 1.000000, val_loss: 0.590903, val_acc: 1.000000 

epoch--:5
Best number of rounds: 4,best_train_accur

KeyboardInterrupt: 

In [5]:
val_summary

NameError: name 'val_summary' is not defined